In [7]:
import random

# ==== RATING CALCULATION (UNCHANGED) ====

def compute_rating(wins, losses, runs_scored, runs_allowed, on_base, slugging):
    games = wins + losses
    # Handle case where games is 0 to prevent ZeroDivisionError
    if games == 0:
        return 0.0
    win_pct = wins / games
    run_diff_per_game = (runs_scored - runs_allowed) / games
    run_diff_norm = (run_diff_per_game + 3) / 6  # Normalize to [0, 1]
    pythag_winloss = (runs_scored ** 1.83) / ((runs_scored ** 1.83) + (runs_allowed ** 1.83))
    # Use ** for exponentiation, not ^
    return (pythag_winloss * 0.35) + (win_pct * 0.1) + (run_diff_norm * 0.3) + (on_base * 0.1) + (slugging * 0.15)


# ==== GAME & SERIES SIMULATION (GAME LOGIC UNCHANGED) ====

def game_win_prob(team_rating, opp_rating, home_field_advantage=0):
    # Handle case where both ratings are 0 to prevent ZeroDivisionError
    if (team_rating + home_field_advantage + opp_rating) == 0:
        return 0.5 # Default to 50/50 if no rating information
    return (team_rating + home_field_advantage) / (team_rating + home_field_advantage + opp_rating)

def simulate_game_100_times(team_rating, opp_rating, is_home, hfa=0.05):
    team_wins = 0
    opp_wins = 0
    for _ in range(100): # Simulate each game 100 times
        prob = game_win_prob(team_rating, opp_rating, hfa if is_home else 0)
        if random.random() < prob:
            team_wins += 1
        else:
            opp_wins += 1
    return team_wins > opp_wins # Return True if team wins more simulations, False otherwise

def get_home_games(round_name, home_team_is_A):
    formats = {
        "wildcard":      [True, True, True],
        "divisional":    [True, True, False, False, True],
        "lcs":           [True, True, False, False, False, True, True],
        "world_series":  [True, True, False, False, False, True, True],
    }
    base = formats[round_name]
    return base if home_team_is_A else [not g for g in base]

def simulate_series(team_A, team_B, round_name, actual_results=None):
    """
    Simulates a series between team_A and team_B.
    actual_results: A list of booleans representing actual game outcomes.
                    True means team_A won the game, False means team_B won.
                    Example: [True] for a single game win by team A.
    """
    if actual_results is None:
        actual_results = []
        
    series_format = {
        "wildcard":      (2, 3),
        "divisional":    (3, 5),
        "lcs":           (4, 7),
        "world_series":  (4, 7)
    }[round_name]

    wins_needed, max_games = series_format

    # HOME FIELD LOGIC (UNCHANGED)
    if round_name == "world_series":
        # For World Series, the home field advantage is determined by the team with the better regular season record.
        # If records are tied, then by seed.
        if team_A["wins"] != team_B["wins"]:
            home_team_is_A = team_A["wins"] > team_B["wins"]
        else:
            home_team_is_A = team_A["seed"] < team_B["seed"]
    else:
        home_team_is_A = team_A["seed"] < team_B["seed"]

    home_games = get_home_games(round_name, home_team_is_A)
    home_team_in_series = team_A if home_team_is_A else team_B
    # away_team_in_series = team_B if home_team_is_A else team_A # Not strictly needed

    print(f"\n🎯 {round_name.upper()} SERIES: {team_A['name']} (Seed {team_A['seed']}) vs {team_B['name']} (Seed {team_B['seed']})")
    print(f"🏟 Home field advantage: {home_team_in_series['name']}")

    # Initialize win counts based on actual results
    team_A_wins = sum(1 for result in actual_results if result is True)
    team_B_wins = sum(1 for result in actual_results if result is False)
    
    # Check for series completion by actual results
    if team_A_wins == wins_needed:
        print(f"✅ Series already concluded by actual results: {team_A['name']} win {team_A_wins}-{team_B_wins}.")
        return team_A
    elif team_B_wins == wins_needed:
        print(f"✅ Series already concluded by actual results: {team_B['name']} win {team_B_wins}-{team_A_wins}.")
        return team_B
    
    # Report actual results that have occurred
    if actual_results:
        print(f"Actual current series score: {team_A['name']} {team_A_wins} - {team_B['name']} {team_B_wins}")

    for i in range(max_games):
        # Ensure 'i' is within the bounds of 'home_games' list
        if i >= len(home_games):
            break

        is_home_for_A = home_games[i]
        
        # Determine which team is home for this specific game
        current_home_team = team_A if is_home_for_A else team_B
        
        # === NEW LOGIC: CHECK FOR ACTUAL RESULT ===
        if i < len(actual_results):
            winner_A = actual_results[i]
            print(f"Game {i+1} (Home: {current_home_team['name']}) - ACTUAL: {'Win' if winner_A else 'Loss'} for {team_A['name']}")
            # Win counts were already set, just continue to check for series win.
        else:
            # === ORIGINAL SIMULATION LOGIC FOR REMAINING GAMES ===
            winner_A = simulate_game_100_times(team_A['rating'], team_B['rating'], is_home_for_A)
            
            if winner_A:
                team_A_wins += 1
                print(f"Game {i+1} (Home: {current_home_team['name']}): {team_A['name']} win.")
            else:
                team_B_wins += 1
                print(f"Game {i+1} (Home: {current_home_team['name']}): {team_B['name']} win.")

        # Check for series win after each game
        if team_A_wins == wins_needed:
            print(f"🏆 {team_A['name']} win the {round_name} series {team_A_wins}-{team_B_wins}")
            return team_A
        elif team_B_wins == wins_needed:
            print(f"🏆 {team_B['name']} win the {round_name} series {team_B_wins}-{team_A_wins}")
            return team_B
            
    # Should not be reached if max_games is correctly set, but included for safety
    if team_A_wins > team_B_wins:
        return team_A
    elif team_B_wins > team_A_wins:
        return team_B
    else:
        # Possible for a tie to occur if series is not played to completion, which shouldn't happen in MLB
        print("Series ended in a draw (unexpected). Returning A.")
        return team_A


# ==== POSTSEASON BRACKET SIMULATION (UPDATED) ====

def simulate_postseason(teams_AL, teams_NL):
    print("\n⚾ SIMULATING MLB POSTSEASON BRACKET\n")

    # --- INPUT ACTUAL WILD CARD GAME RESULTS ---
    # Wild Card is a best-of-3.
    # The actual_results list should contain True if the first team (team_A, the higher seed) won.
    
    # Example: If ALWC1 Game 1: AL Seed 3 (CLE) vs AL Seed 6 (DET).
    ALWC1_actual = [False, True, False]  # CLE lost Game 1, won Game 2, lost Game 3
    # Example: If ALWC2 Game 1: AL Seed 4 (NYY) vs AL Seed 5 (BOS). 
    ALWC2_actual = [False, True, True]  # NYY lost Game 1, won Game 2, won Game 3
    
    # Example: If NLWC1 Game 1: NL Seed 3 (LAD) vs NL Seed 6 (CIN)
    NLWC1_actual = [True, True] # LAD won Game 1 and Game 2
    # Example: If NLWC2 Game 1: NL Seed 4 (CHC) vs NL Seed 5 (SDP).
    NLWC2_actual = [True, False, True] # CHC won Game 1, lost Game 2, won Game 3

    ALDS1_actual = [True, True, False, True] 
    ALDS2_actual = [False, True, True, False, True]
    
    NLDS1_actual = [True, True, False, False, True]
    NLDS2_actual = [False, False, True, False]

    ALCS_actual = [False, False, True, True]
    NLCS_actual = [False, False, False]

    # Wild Card Round
    print("\n--- WILD CARD ROUND ---")
    ALWC1 = simulate_series(teams_AL[3], teams_AL[6], "wildcard", actual_results=ALWC1_actual)
    ALWC2 = simulate_series(teams_AL[4], teams_AL[5], "wildcard", actual_results=ALWC2_actual)
    NLWC1 = simulate_series(teams_NL[3], teams_NL[6], "wildcard", actual_results=NLWC1_actual)
    NLWC2 = simulate_series(teams_NL[4], teams_NL[5], "wildcard", actual_results=NLWC2_actual)

    # Division Series
    print("\n--- DIVISION SERIES ---")
    ALDS1 = simulate_series(teams_AL[1], ALWC2, "divisional", actual_results=ALDS1_actual)
    ALDS2 = simulate_series(teams_AL[2], ALWC1, "divisional", actual_results=ALDS2_actual)
    NLDS1 = simulate_series(teams_NL[1], NLWC2, "divisional", actual_results=NLDS1_actual)
    NLDS2 = simulate_series(teams_NL[2], NLWC1, "divisional", actual_results=NLDS2_actual)

    # League Championship
    print("\n--- LEAGUE CHAMPIONSHIP SERIES ---")
    ALCS = simulate_series(ALDS1, ALDS2, "lcs", actual_results = ALCS_actual)
    NLCS = simulate_series(NLDS1, NLDS2, "lcs", actual_results = NLCS_actual)

    # World Series
    print("\n--- WORLD SERIES ---")
    WS_winner = simulate_series(ALCS, NLCS, "world_series")
    print(f"\n🏁 WORLD SERIES CHAMPION: {WS_winner['name']} 🏆\n")


# ==== TEAM DATA (UNCHANGED) ====

def create_team(name, wins, losses, runs_scored, runs_allowed, on_base, slugging, seed):
    return {
        "name": name,
        "wins": wins,
        "losses": losses,
        "runs_scored": runs_scored,
        "runs_allowed": runs_allowed,
        "on_base": on_base,
        "slugging": slugging,
        "seed": seed,
        "rating": compute_rating(wins, losses, runs_scored, runs_allowed, on_base, slugging)
    }

teams_AL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Toronto Blue Jays", 94, 68, 798, 721, 0.333, 0.427, 1),
    create_team("Seattle Mariners", 90, 72, 766, 694, 0.320, 0.420, 2),
    create_team("Cleveland Guardians", 88, 74, 643, 649, 0.296, 0.373, 3),
    create_team("New York Yankees", 94, 68, 849, 685, 0.332, 0.455, 4),
    create_team("Boston Red Sox", 89, 73, 786, 676, 0.324, 0.421, 5),
    create_team("Detroit Tigers", 87, 75, 758, 691, 0.316, 0.413, 6),
]

teams_NL = [
    None, # 0 index is unused, keeping for 1-based seed indexing
    create_team("Milwaukee Brewers", 97, 65, 806, 634, 0.332, 0.420, 1),
    create_team("Philadelphia Phillies", 96, 66, 778, 648, 0.328, 0.431, 2),
    create_team("Los Angeles Dodgers", 93, 69, 825, 683, 0.327, 0.441, 3),
    create_team("Chicago Cubs", 92, 70, 793, 649, 0.320, 0.430, 4),
    create_team("San Diego Padres", 90, 72, 702, 621, 0.321, 0.390, 5),
    create_team("Cincinnati Reds", 83, 79, 716, 681, 0.315, 0.391, 6),
]

simulate_postseason(teams_AL, teams_NL)


⚾ SIMULATING MLB POSTSEASON BRACKET


--- WILD CARD ROUND ---

🎯 WILDCARD SERIES: Cleveland Guardians (Seed 3) vs Detroit Tigers (Seed 6)
🏟 Home field advantage: Cleveland Guardians
✅ Series already concluded by actual results: Detroit Tigers win 2-1.

🎯 WILDCARD SERIES: New York Yankees (Seed 4) vs Boston Red Sox (Seed 5)
🏟 Home field advantage: New York Yankees
✅ Series already concluded by actual results: New York Yankees win 2-1.

🎯 WILDCARD SERIES: Los Angeles Dodgers (Seed 3) vs Cincinnati Reds (Seed 6)
🏟 Home field advantage: Los Angeles Dodgers
✅ Series already concluded by actual results: Los Angeles Dodgers win 2-0.

🎯 WILDCARD SERIES: Chicago Cubs (Seed 4) vs San Diego Padres (Seed 5)
🏟 Home field advantage: Chicago Cubs
✅ Series already concluded by actual results: Chicago Cubs win 2-1.

--- DIVISION SERIES ---

🎯 DIVISIONAL SERIES: Toronto Blue Jays (Seed 1) vs New York Yankees (Seed 4)
🏟 Home field advantage: Toronto Blue Jays
✅ Series already concluded by actual results